In [1]:
import pandas as pd
import numpy as np
import math

measurement_year = str(2023)

In [2]:
#Reads OMI data

df_omi = pd.read_csv('./OMI-dataprocess-output/omi_'+measurement_year+'.csv',  sep=r',', header=0)
year_omi = df_omi['YEAR']
month_omi = df_omi['MONTH']
day_omi = df_omi['DAY']
doy_omi = df_omi['DOY']
ozone_daily_mean_omi = df_omi['OMI_O3']

#print(df_omi)
o3_mean_omi = df_omi['OMI_O3'].mean()
#print(o3_mean_omi)

maxo3omi = df_omi['OMI_O3'].idxmax()
mino3omi = df_omi['OMI_O3'].idxmin()
print(df_omi.head())

   Unnamed: 0  YEAR  MONTH  DAY  DOY      OMI_O3
0           0  2023      1    1    1         NaN
1           1  2023      1    2    2  282.000000
2           2  2023      1    3    3  268.899994
3           3  2023      1    4    4         NaN
4           4  2023      1    5    5  286.100006


In [3]:
#Reads Look up table data

df_lut = pd.read_csv('../Data/LookUpTable/'+measurement_year+'/summary.115',  sep='\s+',skiprows=22, header=None)
df_lut.columns = ['year','month','day','DOY','O3_LUT','no3','o3sig','dose_rate','clt','max_dose_rate','integrated_daily_dose']

In [4]:
#creating realtiv diff dataframe to merge OMI and Look up table O3 data (if both exist) and calcule absolute and relative differance
df_lut.loc[df_lut['O3_LUT']==-99.9,'O3_LUT'] = 0
diff = pd.merge(df_omi, df_lut, on='DOY')

#dropping missing data points
index_names = diff[ (diff['OMI_O3'].isnull() ) | (diff['O3_LUT']==0.0 )].index
diff.drop(index_names, inplace = True)
diff['Absolute_diff'] = (diff['OMI_O3']-diff['O3_LUT']).abs()
diff['Relative_diff'] = diff['Absolute_diff'] / (diff['OMI_O3'] + diff['O3_LUT'])
diff.reset_index()


,index,Unnamed: 0,YEAR,MONTH,DAY,DOY,OMI_O3,year,month,day,O3_LUT,no3,o3sig,dose_rate,clt,max_dose_rate,integrated_daily_dose,Absolute_diff,Relative_diff
0,1,1,2023,1,2,2,282.000000,2023,1,2,265.2,60,2.3,0.04731,104.66,0.05563,546.3,16.800000,0.030702
1,2,2,2023,1,3,3,268.899994,2023,1,3,270.4,26,2.2,0.01292,28.91,0.02624,217.0,1.500006,0.002781
2,3,4,2023,1,5,5,286.100006,2023,1,5,272.8,60,0.9,0.02320,51.38,0.03241,353.9,13.300006,0.023797
3,5,6,2023,1,7,7,376.799988,2023,1,7,356.5,60,1.4,0.02272,95.25,0.02779,318.3,20.299988,0.027683
4,7,8,2023,1,9,9,328.100006,2023,1,9,312.2,60,2.4,0.04223,127.13,0.04385,592.7,15.900006,0.024832
5,8,9,2023,1,10,10,325.399994,2023,1,10,308.4,60,1.1,0.02010,54.33,0.02212,265.1,16.999994,0.026822
6,9,10,2023,1,11,11,321.100006,2023,1,11,294.4,60,0.9,0.03086,73.99,0.03620,493.3,26.700006,0.043379
7,10,11,2023,1,12,12,330.500000,2023,1,12,315.6,60,2.2,0.01495,40.75,0.02370,199.2,14.900000,0.023061
8,12,13,2023,1,14,14,348.299988,2023,1,14,327.2,60,0.5,0.02820,83.46,0.02968,377.1,21.099988,0.031236
9,14,15,2023,1,16,16,327.899994,2023,1,16,307.5,60,0.4,0.05158,130.71,0.05253,768.7,20.399994,0.032106


In [5]:
omi_o3_mean = diff['OMI_O3'].mean()
omi_o3_std = diff['OMI_O3'].std()
lut_o3_mean = diff['O3_LUT'].mean()
lut_o3_std = diff['O3_LUT'].std()
abs_diff = diff['Absolute_diff'].mean()
rel_diff = diff['Relative_diff'].mean()
measured_days = len(diff['Absolute_diff'])
print("measured days: "+str(measured_days))
print("OMI daily mean: "+str(omi_o3_mean))
print("OMI daily std: "+str(omi_o3_std))
print("LUT daily mean: "+str(lut_o3_mean))
print("LUT daily std: "+str(lut_o3_std))
print("Absolute diff: " + str(abs_diff))
print("Relative diff: " + str(rel_diff))


measured days: 38
OMI daily mean: 323.28157766241776
OMI daily std: 37.94579228136818
LUT daily mean: 315.75000000000006
LUT daily std: 35.34166632147356
Absolute diff: 13.073686137952308
Relative diff: 0.020536317501598175
